In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import cv2
import PIL
import os
from tqdm import tqdm
import copy
import shutil

import torchvision.models as models
from torchvision.transforms import transforms

import torch
import torch.nn as nn 
from torch.utils.data import DataLoader, Dataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# Dataset

In [24]:
metadata = pd.read_csv("/kaggle/input/metadata/metadata.csv")
metadata = metadata.reset_index(drop=True)

In [25]:
print(len(metadata))
print(metadata.columns)
print(metadata.shape)

18875
Index(['binary_label', 'id', 'smoke', 'drink', 'background_father',
       'background_mother', 'age', 'gender', 'skin_cancer_history',
       'cancer_history', 'region', 'itch', 'grew', 'hurt', 'changed', 'bleed',
       'elevation', 'biopsed', 'fitzpatrick'],
      dtype='object')
(18875, 19)


In [26]:
metadata["id"].loc[16572:16579]

16572    cd90e491ddaa92f0f4eb07f73aa09f64
16573    5f1ed6de6a9110d7dc580a6a0312af63
16574    f198aaf1f0550c2464b285454d34926e
16575    6214de2e915835014235a1839cbc5938
16576    5a3a4c1f0effb626b298e89c032b1d28
16577               PAT_1516_1765_530.png
16578                  PAT_46_881_939.png
16579               PAT_1545_1867_547.png
Name: id, dtype: object

In [27]:
metadata.binary_label.value_counts()

binary_label
0    14793
1     4082
Name: count, dtype: int64

In [28]:
metadata.fitzpatrick.value_counts()

fitzpatrick
 2.0    5684
 3.0    3700
 1.0    3100
 4.0    2843
 5.0    1543
-1.0    1369
 6.0     636
Name: count, dtype: int64

In [29]:
metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18875 entries, 0 to 18874
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   binary_label         18875 non-null  int64  
 1   id                   18875 non-null  object 
 2   smoke                18875 non-null  int64  
 3   drink                18875 non-null  int64  
 4   background_father    18875 non-null  int64  
 5   background_mother    18875 non-null  int64  
 6   age                  18875 non-null  float64
 7   gender               18875 non-null  int64  
 8   skin_cancer_history  18875 non-null  int64  
 9   cancer_history       18875 non-null  int64  
 10  region               18875 non-null  int64  
 11  itch                 18875 non-null  int64  
 12  grew                 18875 non-null  int64  
 13  hurt                 18875 non-null  int64  
 14  changed              18875 non-null  int64  
 15  bleed                18875 non-null 

In [30]:
pad_base_path = "/kaggle/input/skin-cancer"
pad_subfolder_paths = [
    "imgs_part_1/imgs_part_1",
    "imgs_part_2/imgs_part_2",
    "imgs_part_3/imgs_part_3"
]
pad_set = {}

for pad_path in pad_subfolder_paths:
    image_path = os.path.join(pad_base_path, pad_path)
        
    for f in os.listdir(image_path):
        if f.endswith(".png"):
            pad_set[f] = os.path.join(image_path, f)

In [31]:
def resolve_path(image_id):
    if image_id.endswith(".png"):
        return pad_set.get(image_id, None)
    else:
        return image_id

In [32]:
metadata["full_path"] = metadata["id"].apply(resolve_path)

In [33]:
metadata.full_path

0                         5e82a45bc5d78bd24ae9202d194423f8
1                         fa2911a9b13b6f8af79cb700937cc14f
2                         d2bac3c9e4499032ca8e9b07c7d3bc40
3                         0a94359e7eaacd7178e06b2823777789
4                         a39ec3b1f22c08a421fa20535e037bba
                               ...                        
18870    /kaggle/input/skin-cancer/imgs_part_3/imgs_par...
18871    /kaggle/input/skin-cancer/imgs_part_1/imgs_par...
18872    /kaggle/input/skin-cancer/imgs_part_3/imgs_par...
18873    /kaggle/input/skin-cancer/imgs_part_1/imgs_par...
18874    /kaggle/input/skin-cancer/imgs_part_3/imgs_par...
Name: full_path, Length: 18875, dtype: object

In [34]:
metadata_cols = ['smoke', 'drink', 'background_father',
       'background_mother', 'age', 'gender', 'skin_cancer_history',
       'cancer_history', 'region', 'itch', 'grew', 'hurt', 'changed', 'bleed',
       'elevation', 'biopsed', 'fitzpatrick']

In [35]:
class MultimodalSkinCancerDataset(Dataset):

    def __init__(self, df, metadata_cols, transform = None):
        self.df = df
        self.transform = transform
        self.metadata_cols = metadata_cols
       
    
    def __len__(self):
        return len(self.df)


    def __getitem__(self, idx):

        image = None
        image_path = None
        fitz_path = "/kaggle/input/fitzpatrick17k-original/finalfitz17k"

        row = self.df.iloc[idx]
        image_id = self.df.iloc[idx]["id"]
        label = self.df.iloc[idx]["binary_label"]

        
        if(image_id.endswith(".jpg")):
            image_path = os.path.join(fitz_path, image_id)
            image_bgr = cv2.imread(image_path)
            
        elif image_id.endswith(".png"):  
            image_path = self.df.iloc[idx]["full_path"]
        
        else:
            image_path = os.path.join(fitz_path, image_id + ".jpg")
            
        image_bgr = cv2.imread(image_path)

        if image_bgr is None:
            image_bgr = np.zeros((224, 224, 3), dtype=np.uint8)
        
        image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
        
        if self.transform:
            image = self.transform(image)

        metadata = row[self.metadata_cols].values.astype(np.float32)
            
        return image, metadata, label

In [36]:
image_size = 224  

train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((image_size, image_size)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

val_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [37]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

train_df, val_df = train_test_split(metadata, test_size=0.2, random_state=42, stratify=metadata['binary_label'])

train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

train_dataset = MultimodalSkinCancerDataset(df = train_df, metadata_cols=metadata_cols, transform=train_transform)
val_dataset = MultimodalSkinCancerDataset(df = val_df, metadata_cols=metadata_cols, transform=val_transform)

batch_size = 32

In [38]:
from torch.utils.data import WeightedRandomSampler

train_class_counts = train_df['binary_label'].value_counts().to_dict()
train_weights = [1.0 / train_class_counts[label] for label in train_df['binary_label']]

sampler = WeightedRandomSampler(train_weights, num_samples=len(train_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Helper Functions

In [39]:
from tqdm import tqdm

def train_one_epoch(model, loader, optimizer, criterion, device):
    
    model.train()
    running_loss = 0
    
    for images, metadata, labels in tqdm(loader, desc="Training", leave=True):
        
        images = images.to(device)
        metadata = metadata.to(device)
        labels = labels.float().unsqueeze(1).to(device)

        optimizer.zero_grad()
        outputs = model(images, metadata)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    return running_loss / len(loader)

In [40]:
def validate(model, loader, criterion, device):
    model.eval()
    running_loss = 0
    all_preds, all_labels = [], []
    
    with torch.no_grad():
        for images, metadata, labels in tqdm(loader, desc="Validation", leave=True):
            images = images.to(device)
            metadata = metadata.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            
            outputs = model(images, metadata)  
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            
            probs = torch.sigmoid(outputs)  
            all_preds.extend(probs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            
    return running_loss / len(loader), all_preds, all_labels

In [41]:
def compute_metrics(preds, labels):
    preds_bin = (np.array(preds) > 0.5).astype(int)
    labels = np.array(labels)
    
    acc = accuracy_score(labels, preds_bin)
    prec = precision_score(labels, preds_bin)
    rec = recall_score(labels, preds_bin)
    f1 = f1_score(labels, preds_bin)
    auc = roc_auc_score(labels, preds)
    
    return {'accuracy': acc, 'precision': prec, 'recall': rec, 'f1': f1, 'roc_auc': auc}

# Modeling

### 1. Resnet18

In [42]:
class MultimodalModel(nn.Module):
    def __init__(self, num_metadata_features, pretrained=True):
        super().__init__()
        
        self.cnn = models.resnet18(pretrained=pretrained)
        self.cnn.fc = nn.Identity()
        img_features = 512  
        
        self.metadata_fc = nn.Sequential(
            nn.Linear(num_metadata_features, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(img_features + 32, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)  
        )
    
    def forward(self, image, metadata=None):
        img_out = self.cnn(image)
        if metadata is not None:
            meta_out = self.metadata_fc(metadata)
            combined = torch.cat([img_out, meta_out], dim=1)
        else:
            combined = img_out
        return self.classifier(combined) 

In [43]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MultimodalModel(num_metadata_features=len(train_dataset.metadata_cols)).to(device)

class_counts = metadata['binary_label'].value_counts()
neg_count, pos_count = class_counts[0], class_counts[1]
pos_weight = torch.tensor([neg_count / pos_count], dtype=torch.float32).to(device)

criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [44]:
num_epochs = 10
best_val_f1 = 0.0  
best_val_loss = float('inf') 
patience = 5
epochs_no_improve = 0

for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_preds, val_labels = validate(model, val_loader, criterion, device)
    metrics = compute_metrics(val_preds, val_labels)

    print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    print(f"Metrics: {metrics}")

    if metrics['f1'] > best_val_f1:
        best_val_f1 = metrics['f1']
        torch.save(model.state_dict(), "best_multimodal_model.pth")
        print("Model saved! F1-score improved.")
        epochs_no_improve = 0 
    
    else:
        epochs_no_improve += 1
        print(f"F1-score did not improve for {epochs_no_improve} epochs.")

    if epochs_no_improve == patience:
        print(f"Early stopping triggered after {patience} epochs with no improvement.")
        break

Validation: 100%|██████████| 118/118 [00:22<00:00,  5.14it/s]


Epoch 1/10 | Train Loss: 0.6570 | Val Loss: 0.6176
Metrics: {'accuracy': 0.7952317880794701, 'precision': 0.5153024911032028, 'recall': 0.8872549019607843, 'f1': 0.6519585772174696, 'roc_auc': 0.919701608253981}
Model saved! F1-score improved.


Validation: 100%|██████████| 118/118 [00:22<00:00,  5.17it/s]


Epoch 2/10 | Train Loss: 0.4569 | Val Loss: 0.5118
Metrics: {'accuracy': 0.8770860927152317, 'precision': 0.6651031894934334, 'recall': 0.8688725490196079, 'f1': 0.7534537725823591, 'roc_auc': 0.9431072699441387}
Model saved! F1-score improved.


Validation: 100%|██████████| 118/118 [00:22<00:00,  5.14it/s]


...

Epoch 8/10 | Train Loss: 0.1695 | Val Loss: 0.7129
Metrics: {'accuracy': 0.9149668874172185, 'precision': 0.8136882129277566, 'recall': 0.7867647058823529, 'f1': 0.7999999999999999, 'roc_auc': 0.9468760975157214}
Model saved! F1-score improved.


Validation: 100%|██████████| 118/118 [00:22<00:00,  5.14it/s]


Epoch 9/10 | Train Loss: 0.1563 | Val Loss: 0.6101
Metrics: {'accuracy': 0.909933774834437, 'precision': 0.7644444444444445, 'recall': 0.8431372549019608, 'f1': 0.8018648018648018, 'roc_auc': 0.9551412192115779}
Model saved! F1-score improved.


Validation: 100%|██████████| 118/118 [00:22<00:00,  5.16it/s]


Epoch 10/10 | Train Loss: 0.1429 | Val Loss: 0.6020
Metrics: {'accuracy': 0.9101986754966888, 'precision': 0.7647058823529411, 'recall': 0.8443627450980392, 'f1': 0.8025626092020967, 'roc_auc': 0.9567367585763608}
Model saved! F1-score improved.


In [45]:
model.load_state_dict(torch.load("best_multimodal_model.pth"))

<All keys matched successfully>